# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [ ]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [3]:
'''
1.	Недостатки текущей реализации:
	1	Поддерживаются только операции сложения и умножения.
	2	Отсутствует поддержка унарных операций (например, отрицание).
	3	Нет поддержки деления и возведения в степень.
	4	Ограничена поддержка функций одной переменной
'''

from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float  # Значение
    d: float      # Производная

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        # Реализация сложения
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        # Реализация вычитания
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        # Реализация умножения
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        # Реализация деления
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value ** 2))
            case Number():
                return Dual(self.value / float(other), self.d / float(other))

    def __pow__(self, power: Union[int, float]) -> "Dual":
        # Реализация возведения в степень
        return Dual(self.value ** power, self.d * power * (self.value ** (power - 1)))

    def __neg__(self) -> "Dual":
        # Реализация унарного отрицания
        return Dual(-self.value, -self.d)

    __rmul__ = __mul__  # Реализация обратного умножения
    __radd__ = __add__  # Реализация обратного сложения
    __rsub__ = __sub__  # Реализация обратного вычитания

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    # Возвращает функцию, вычисляющую производную
    return lambda x: func(Dual(x, 1.0)).d 

# Тестовая функция для дифференцирования
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# Запуск тестов
def run_tests():
    print("Запуск тестов...")

    # Тестирование производной
    assert f_diff(2) == 22, "Тест производной не пройден"

    # Тестирование вычитания
    a = Dual(5.0, 1.0)
    b = Dual(3.0, 1.0)
    assert (a - b).value == 2.0 and (a - b).d == 0.0, "Тест вычитания не пройден"

    # Тестирование деления
    assert (a / b).value == 5.0 / 3.0, "Тест деления не пройден"

    # Тестирование возведения в степень
    assert (a ** 2).value == 25.0 and (a ** 2).d == 2 * 5.0 * 1.0, "Тест возведения в степень не пройден"

    # Тестирование унарного отрицания
    assert (-a).value == -5.0 and (-a).d == -1.0, "Тест унарного отрицания не пройден"

    print("Все тесты успешно пройдены!")

# Запуск всех тестов
run_tests()



## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [4]:
import math
from dataclasses import dataclass

@dataclass
class Dual:
    value: float  # Значение
    d: float      # Производная

    def exp(self) -> "Dual":
        # Вычисляем e^x и его производную
        return Dual(math.exp(self.value), math.exp(self.value) * self.d)

    def sin(self) -> "Dual":
        # Вычисляем sin(x) и его производную
        return Dual(math.sin(self.value), math.cos(self.value) * self.d)

    def cos(self) -> "Dual":
        # Вычисляем cos(x) и его производную
        return Dual(math.cos(self.value), -math.sin(self.value) * self.d)

    def log(self) -> "Dual":
        # Проверяем, что значение больше нуля перед вычислением логарифма
        if self.value <= 0:
            raise ValueError("Вход для логарифма должен быть больше нуля")
        return Dual(math.log(self.value), self.d / self.value)

def test_functions():
    print("Запуск тестов...")
    
    x = Dual(1, 1)
    
    # Проверка функции exp
    assert round(x.exp().value, 5) == round(math.exp(1), 5), "Тест exp не пройден"
    
    # Проверка функции sin
    assert round(x.sin().value, 5) == round(math.sin(1), 5), "Тест sin не пройден"
    
    # Проверка функции cos
    assert round(x.cos().value, 5) == round(math.cos(1), 5), "Тест cos не пройден"

    # Проверка функции log
    assert round(x.log().value, 5) == round(math.log(1), 5), "Тест log не пройден"
    
    # Добавление других тестовых случаев
    y = Dual(2, 1)
    assert round(y.exp().value, 5) == round(math.exp(2), 5), "Тест exp для 2 не пройден"
    assert round(y.sin().value, 5) == round(math.sin(2), 5), "Тест sin для 2 не пройден"
    assert round(y.cos().value, 5) == round(math.cos(2), 5), "Тест cos для 2 не пройден"
    
    print("Все тесты успешно пройдены!")

# Запуск тестов
test_functions()




## Задание 1.3 (2 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+1 балл**)

In [5]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [6]:

import numpy as np
from scipy.optimize import approx_fprime

def test_numerical_derivative():
    # Определяем функцию для тестирования
    f = lambda x: 5 * x * x + 2 * x + 2
    # Вычисляем производную с использованием функции diff
    f_diff = diff(f)

    # Используем approx_fprime для вычисления численной производной
    numerical_derivative = approx_fprime(np.array([2.0]), f, epsilon=1e-6)[0]

    # Проверяем, совпадают ли автоматическое дифференцирование и численная производная
    assert round(f_diff(2), 5) == round(numerical_derivative, 5), "Тест производной не пройден"

# Запуск теста
test_numerical_derivative()



## Задание 1.4 (6 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
'''
Можно использовать стандартный модуль ast для генерации случайных функций и их проверки.

Пример генерации функции и проверки её производной:
'''
import ast
import random
import numpy as np
from scipy.misc import derivative  # 需要确保在环境中导入
from typing import Callable

def random_function() -> Callable[[float], float]:
    # Генерация случайной функции с использованием ast
    expr = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[ast.arg(arg='x')],
                posonlyargs=[],
                kwonlyargs=[], kw_defaults=[], defaults=[]
            ),
            # Создаем бинарные операции с случайными значениями
            body=ast.BinOp(
                left=ast.BinOp(
                    left=ast.Constant(value=random.randint(1, 10)),
                    op=ast.Mult(),
                    right=ast.Name(id='x', ctx=ast.Load())
                ),
                op=random.choice([ast.Add(), ast.Sub()]),
                right=ast.Constant(value=random.randint(1, 10))
            )
        )
    )
    ast.fix_missing_locations(expr)
    return eval(compile(expr, filename="", mode="eval"))

def test_random_functions():
    for _ in range(10):
        func = random_function()  # Генерируем случайную функцию
        f_diff = diff(func)       # Вычисляем производную автоматическим методом
        x = random.uniform(0, 10) # Генерируем случайное значение x
        
        # Проверяем, совпадают ли производные
        assert round(f_diff(x), 5) == round(derivative(func, x), 5), \
            f"Ошибка в функции: {func} для x = {x}"

# Запуск тестов
test_random_functions()


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:
import math
from dataclasses import dataclass
from typing import Union, Callable, List

@dataclass
class Dual:
    value: float  # Значение
    d: List[float]  # Производные (список для поддержки нескольких аргументов)

    def __add__(self, other: Union["Dual", float]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                # Реализация сложения для объектов Dual
                return Dual(self.value + o_value, [self.d[i] + o_d[i] for i in range(len(self.d))])
            case float():
                # Реализация сложения для числа
                return Dual(self.value + other, self.d)

    def __sub__(self, other: Union["Dual", float]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                # Реализация вычитания для объектов Dual
                return Dual(self.value - o_value, [self.d[i] - o_d[i] for i in range(len(self.d))])
            case float():
                # Реализация вычитания для числа
                return Dual(self.value - other, self.d)

    def __mul__(self, other: Union["Dual", float]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                # Реализация умножения для объектов Dual
                return Dual(self.value * o_value, [self.value * o_d[i] + self.d[i] * o_value for i in range(len(self.d))])
            case float():
                # Реализация умножения для числа
                return Dual(self.value * other, [self.d[i] * other for i in range(len(self.d))])

    def __truediv__(self, other: Union["Dual", float]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                # Реализация деления для объектов Dual
                return Dual(self.value / o_value, [(self.d[i] * o_value - self.value * o_d[i]) / (o_value ** 2) for i in range(len(self.d))])
            case float():
                # Реализация деления для числа
                return Dual(self.value / other, [d / other for d in self.d])

    def __pow__(self, power: Union[int, float]) -> "Dual":
        # Реализация возведения в степень
        return Dual(self.value ** power, [d * power * (self.value ** (power - 1)) for d in self.d])

    def __neg__(self) -> "Dual":
        # Реализация унарного отрицания
        return Dual(-self.value, [-d for d in self.d])


def diff(func: Callable[..., float]) -> Callable[..., List[float]]:
    # Возвращает функцию, вычисляющую производную
    def wrapper(*args: float) -> List[float]:
        # Создаем объекты Dual с производными по единице для каждого аргумента
        duals = [Dual(arg, [1.0 if i == j else 0.0 for j in range(len(args))]) for i, arg in enumerate(args)]
        
        # Вычисляем значение функции с использованием объектов Dual
        result = func(*duals)
        
        # Возвращаем только производные
        return [dual.d[0] for dual in duals]

    return wrapper

# Пример функции для тестирования
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

# Автоматическое дифференцирование функции f
f_diff = diff(f)

# Тестирование производной в определенной точке x=10, y=10, z=10
print(f_diff(10, 10, 10))  # Ожидаемый результат: [10, 5, 1]

